In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install fuzzywuzzy
!pip install surprise

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from ast import literal_eval
from surprise import SVD, SVDpp, KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate,train_test_split, GridSearchCV
from surprise import NormalPredictor
from surprise import Reader

import re 
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')

/opt/conda/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import lightgbm as lgb
import warnings; warnings.simplefilter('ignore')

In [3]:
#temp_train = pd.read_csv('train.csv')

In [4]:
#temp_test = pd.read_csv('test.csv')

In [5]:
#temp_test['id'].isna().sum()
#test_features[test_features.duplicated('msno')==True].shape

In [6]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')

In [7]:
# temp = temp_train[['target']]
# type(temp)

In [8]:
# temp2 = temp_test[['id','msno']]

In [9]:
train_features.shape

(7377418, 37)

In [9]:
#train_features = pd.concat([train_features, temp])

In [10]:
# del temp
# del temp_train

In [24]:
# #test_features = pd.merge(test_features, temp2, on='msno',how='inner')
# test_ = pd.concat([df1, df4], axis=1, join_axes=[df1.index])

In [10]:
test_features.head(5)

,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length_x,genre_ids_x,...,language_y,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,224130.0,458,...,3.0,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0
1,1,1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,320470.0,465,...,3.0,3804.0,6090.0,62.5,67621.0,115325.0,52.0,142.0,36.6,2.0
2,2,2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist,315899.0,2022,...,17.0,2.0,5.0,40.0,286.0,989.0,19.0,132.0,14.4,2.0
3,3,3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio,285210.0,465,...,52.0,7.0,31.0,22.6,228.0,698.0,115.0,388.0,29.6,0.0
4,4,4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio,197590.0,873,...,-1.0,2.0,5.0,40.0,59.0,180.0,115.0,388.0,29.6,2.0


In [11]:
train_features.head(1)

,Unnamed: 0,Unnamed: 0.1,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length_x,genre_ids_x,...,language_y,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year
0,0,0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,...,52.0,102,215,47.4,528.0,1140,2791,5511,50.6,3.0


In [12]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

np.set_printoptions(threshold=np.inf)

In [13]:
train_features.drop('song_length_y',axis = 1 , inplace =True)
train_features.drop('genre_ids_y',axis = 1 , inplace =True)
train_features.drop('artist_name_y',axis = 1 , inplace =True)
train_features.drop('composer_y',axis = 1 , inplace =True)
train_features.drop('lyricist_y',axis = 1 , inplace =True)
train_features.drop('language_y',axis = 1 , inplace =True)

test_features.drop('song_length_y',axis = 1 , inplace =True)
test_features.drop('genre_ids_y',axis = 1 , inplace =True)
test_features.drop('artist_name_y',axis = 1 , inplace =True)
test_features.drop('composer_y',axis = 1 , inplace =True)
test_features.drop('lyricist_y',axis = 1 , inplace =True)
test_features.drop('language_y',axis = 1 , inplace =True)

In [14]:
train_features.drop('lyricist_x',axis = 1 , inplace =True)
test_features.drop('lyricist_x',axis = 1 , inplace =True)

train_features.drop('source_type',axis = 1 , inplace =True)
test_features.drop('source_type',axis = 1 , inplace =True)


In [15]:
test_features.head(1)

,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,song_length_x,genre_ids_x,artist_name_x,composer_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,3.0,愛其實很殘忍,TWUM71400047,1,0,NaN,7,2016-02-19,2017-09-18,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0


In [16]:
def keepwords(text):
    temp = re.sub('[^a-zA-Z]', ' ', str(text))
    return temp.lower() + ' '

#train_features['name'] = train_features['name'].astype('category')
train_features['name'] = train_features.name.apply(lambda x: keepwords(x))
test_features['name'] = test_features.name.apply(lambda x: keepwords(x))

#train_features['artist_name_x'] = train_features['artist_name_x'].astype('category')
train_features['artist_name_x'] = train_features.artist_name_x.apply(lambda x: keepwords(x))
test_features['artist_name_x'] = test_features.artist_name_x.apply(lambda x: keepwords(x))

#train_features['source_screen_name'] = train_features['source_screen_name'].astype('category')
train_features['source_screen_name'] = train_features.source_screen_name.apply(lambda x: keepwords(x))
test_features['source_screen_name'] = test_features.source_screen_name.apply(lambda x: keepwords(x))

train_features['composer_x'] = train_features.composer_x.apply(lambda x: keepwords(x))
test_features['composer_x'] = test_features.composer_x.apply(lambda x: keepwords(x))



In [17]:
train_features['description'] = train_features['name'] + train_features['artist_name_x']
test_features['description'] = test_features['name'] + test_features['artist_name_x']
#train_features['description'] = train_features['description'].fillna('')
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')

In [14]:
train_features.shape

(837941, 32)

In [15]:
num = train_features.name.isna().sum()
num

0

In [16]:
train_features.head(10)

,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length_x,genre_ids_x,artist_name_x,composer_x,lyricist_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,description
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,local playlist more,local-library,224130.0,458,rachel liang,Qi Zheng Zhang,NaN,3.0,,TWUM71400047,1,0,NaN,7,2016-02-19,2017-09-18,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0,rachel liang
1,1,1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,local playlist more,local-library,320470.0,465,jj lin,林俊傑,孫燕姿/易家揚,3.0,,TWB671005201,1,0,NaN,7,2016-02-19,2017-09-18,3804.0,6090.0,62.5,67621.0,115325.0,52.0,142.0,36.6,2.0,jj lin
2,2,2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,nan,song-based-playlist,315899.0,2022,yu takahashi,Yu Takahashi,Yu Takahashi,17.0,subarashiki nichijo,JPWP01070260,1,0,NaN,4,2016-11-17,2016-11-24,2.0,5.0,40.0,286.0,989.0,19.0,132.0,14.4,2.0,subarashiki nichijo yu takahashi
3,3,3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,radio,radio,285210.0,465,u,The Edge| Adam Clayton| Larry Mullen| Jr.,NaN,52.0,hold me thrill me kiss me kill me,GBAAN0201228,3,30,male,9,2007-07-25,2017-04-30,7.0,31.0,22.6,228.0,698.0,115.0,388.0,29.6,0.0,hold me thrill me kiss me kill me u
4,4,4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,radio,radio,197590.0,873,yoga mr sound,Neuromancer,NaN,-1.0,om yoga,ITO101121898,3,30,male,9,2007-07-25,2017-04-30,2.0,5.0,40.0,59.0,180.0,115.0,388.0,29.6,2.0,om yoga yoga mr sound
5,5,5,5,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,NV3nhEcMqsawwvSNTUAt9IVAexHLOm0lDfrHyEfN5B0=,radio,radio,radio,216372.0,465,lee hi,NaN,NaN,31.0,,TWA531300295,3,30,male,9,2007-07-25,2017-04-30,32.0,81.0,39.5,1895.0,3733.0,115.0,388.0,29.6,2.0,lee hi
6,6,6,6,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,DPM6G9RB5QO2dvAVPyc70gxHdvu872IjTUvV6LJT8ho=,radio,radio,radio,200306.0,465,boa,Joe Belmaati,BoA / Joe Belmaati,31.0,the shadow,KRA301200363,3,30,male,9,2007-07-25,2017-04-30,34.0,90.0,37.8,1804.0,4425.0,115.0,388.0,29.6,2.0,the shadow boa
7,7,7,7,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,JGXB3PHc0CX0JskwwjmYP8i318BLo7DhChgEj6Yqjt8=,my library,local playlist more,local-playlist,344305.0,458,steve chou,周傳雄,陳信榮,3.0,,TWA470053008,3,30,male,9,2007-07-25,2017-04-30,628.0,1053.0,59.6,2235.0,4204.0,115.0,388.0,29.6,0.0,steve chou
8,8,8,8,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,507plIkmke1jh3wMrHqKore82pPFozADwydR8P0Gx2Q=,radio,radio,radio,289332.0,465,ivy yan,嚴藝丹,嚴藝丹,3.0,,CNR211200331,3,30,male,9,2007-07-25,2017-04-30,546.0,848.0,64.4,878.0,1485.0,115.0,388.0,29.6,2.0,ivy yan
9,9,9,9,3ZQ6oGfcSiUoCrtBPKGa8hHCiFh5jqtDqPVDUl/zrjU=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,local playlist more,local-library,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,1,0,NaN,7,2015-12-19,2017-09-18,1424.0,2381.0,59.8,35317.0,61927.0,224.0,313.0,71.6,2.0,fantastic baby bigbang


In [15]:
import gc
gc.collect()

86

In [18]:
TFidvectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                                 min_df=2, stop_words='english',
                                 use_idf=True)
tf_vector = TfidfVectorizer()

transformer = TfidfTransformer()



In [19]:
tfidf = transformer.fit_transform(tf_vector.fit_transform(train_features.description))

#tf_matrix = tf.fit_transform(train_features['description'])

In [20]:
tfidf.shape

(837941, 32348)

In [21]:
#del train_features

In [22]:
# train_features.drop('name',axis = 1 , inplace =True)
# train_features.drop('artist_name_x',axis = 1 , inplace =True)
# train_features.drop('source_screen_name',axis = 1 , inplace =True)

In [23]:
#np.save("ft_matrix.npy",tf_matrix)

In [24]:
#del tf_matrix

In [25]:
#tf_matrix = np.load('/kaggle/working/ft_matrix.npy', allow_pickle=True)

In [26]:
chunk_size = 500 
matrix_len = tfidf.shape[0] # Not sparse numpy.ndarray
cosine_sim = np.array([])  # to keep the final result of similarities
cosine_sim_list = []
def similarity_cosine_by_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=tfidf[start:end], Y=tfidf[start:end]) # scikit-learn function

for chunk_start in range(0, matrix_len, chunk_size):
    cosine_similarity_chunk = similarity_cosine_by_chunk(chunk_start, chunk_start + chunk_size)
    if cosine_sim.shape[0] == 0:
        #cosine_sim =  np.copy(cosine_similarity_chunk)
        cosine_sim_list.append(cosine_similarity_chunk)
        del cosine_similarity_chunk
    else:
        #cosine_sim =  np.concatenate((cosine_sim,cosine_similarity_chunk), axis=0)
        cosine_sim_list.append(cosine_similarity_chunk)
        del cosine_similarity_chunk

print(cosine_sim.shape)

(0,)


In [27]:
#Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. 
#Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.
#cosine_sim = linear_kernel(tf_matrix, tf_matrix)
#cosine_sim = cosine_similarity(tfidf, tfidf)


In [28]:
len(cosine_sim_list)

1676

In [29]:
train_features = train_features.reset_index()
titles = train_features['name']
indices = pd.Series(train_features.index, index = train_features['description'])

In [30]:
indices.shape


(837941,)

In [31]:
specific_songs = train_features.loc[train_features.name.str.contains('fantastic baby')]
#specific_songs.sort_values("repeat_percentage",inplace=False)
#specific_songs
#这里有多个结果，多个歌曲来源不同的意思吧，比如歌曲版本也是个问题，

In [32]:
dd = specific_songs.head(10)
dd

,index,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length_x,genre_ids_x,artist_name_x,composer_x,lyricist_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,description
9,9,9,9,9,3ZQ6oGfcSiUoCrtBPKGa8hHCiFh5jqtDqPVDUl/zrjU=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,local playlist more,local-library,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,1,0,NaN,7,2015-12-19,2017-09-18,1424.0,2381.0,59.8,35317.0,61927.0,224.0,313.0,71.6,2.0,fantastic baby bigbang
5629,5629,5629,5629,5629,kN5BssYfZDyQnTN6xfjTXImtxjUvqKAhQV/QDpSDMo8=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,discover,nan,top-hits-for-artist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,1,19,NaN,9,2017-01-27,2017-08-03,1424.0,2381.0,59.8,35317.0,61927.0,28.0,56.0,50.0,2.0,fantastic baby bigbang
6012,6012,6012,6012,6012,ocW8JZ9FbC1jxN1Xwzkano19/65RSxAlAUyNHbt/FZY=,A6UBwgZxp37zPPqraIlKNYTuGTsqCp8uIio3UlLwzwQ=,discover,album more,album,232489.0,465,bigbang,Teddy | G-Dragon,G-Dragon | Teddy,31.0,fantastic baby,MYTP11200494,1,0,NaN,9,2013-05-27,2017-10-01,106.0,217.0,48.8,35317.0,61927.0,221.0,703.0,31.4,2.0,fantastic baby bigbang
7543,7543,7543,7543,7543,NGBFuLGTCYZyIikX2bKpI2Z+zO7ILujTyuF5YUOqFzQ=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,discover,online playlist more,online-playlist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,1,0,NaN,7,2016-04-01,2017-09-30,1424.0,2381.0,59.8,35317.0,61927.0,392.0,729.0,53.8,2.0,fantastic baby bigbang
10136,10136,10136,10136,10136,qr0OOeGMpBDaMIhEIKb56TpUy3ca68f/MmgkS62VqbM=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,explore,explore,online-playlist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,13,36,male,9,2007-06-09,2017-09-17,1424.0,2381.0,59.8,35317.0,61927.0,483.0,629.0,76.8,2.0,fantastic baby bigbang
20616,20616,20616,20616,20616,OuKt6XMXwbVdi77iAbrlfWmPYaxRXQNyOn75vojlyeo=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,explore,explore,online-playlist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,1,0,NaN,7,2013-06-26,2017-09-30,1424.0,2381.0,59.8,35317.0,61927.0,260.0,509.0,51.1,2.0,fantastic baby bigbang
22901,22901,22901,22901,22901,bbfIhmUDTvUpESrF1R/kBzF/jPYgWmVHrwP7QFz3VYE=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,local playlist more,local-playlist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,15,25,male,9,2010-05-15,2017-10-04,1424.0,2381.0,59.8,35317.0,61927.0,420.0,544.0,77.2,2.0,fantastic baby bigbang
28342,28342,28342,28342,28342,SBvF5I24a5t1tLIUmtbiTBvh8TcTA6j4Ao/+2UrzsJQ=,A6UBwgZxp37zPPqraIlKNYTuGTsqCp8uIio3UlLwzwQ=,my library,local playlist more,local-playlist,232489.0,465,bigbang,Teddy | G-Dragon,G-Dragon | Teddy,31.0,fantastic baby,MYTP11200494,13,25,female,9,2009-10-24,2017-11-27,106.0,217.0,48.8,35317.0,61927.0,361.0,901.0,40.1,2.0,fantastic baby bigbang
31261,31261,31261,31261,31261,VArBcNT1YfeT77mPz5n2TFIPy3qgvsBiVlZs8zzNu14=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,local playlist more,local-playlist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,1,0,NaN,7,2016-04-03,2017-10-02,1424.0,2381.0,59.8,35317.0,61927.0,167.0,271.0,61.6,2.0,fantastic baby bigbang
33719,33719,33719,33719,33719,9Batxkm3d9lLPBoKvuvWkkxTKCD5AyeQs44uAHW9oTc=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,nan,top-hits-for-artist,232176.0,465,bigbang,NaN,NaN,31.0,fantastic baby,SGA051200112,5,46,female,9,2009-01-09,2017-07-22,1424.0,2381.0,59.8,35317.0,61927.0,340.0,616.0,55.2,2.0,fantastic baby bigbang


In [33]:
#sim_scores = list(enumerate(cosine_sim_list[0][3]))

In [34]:
#sim_scores

In [35]:


    
def get_index_by_name(name):
    specific_songs = train_features.loc[train_features.name.str.contains(name)]
    dd = specific_songs.head(1)
    return dd.index

def get_scores(slist):
    res = []
    for i in slist:
        if i != 0:
            res.append(i)
    return res

def get_cosine_sim_scores(idx):
    res = []
    for i in range(len(cosine_sim_list)):
        temp = cosine_sim_list[i][idx]
        #if len(temp) != 0:
        res.append(temp)
        #temp = map(lambda x: get_scores(x), cosine_sim_list)
    sim_scores = list(enumerate(res))
    #print(sim_scores)
    return sim_scores
        
def get_recommendations2(title):
    idx = get_index_by_name(title)
    sim_scores = get_cosine_sim_scores(idx)
    sim_scores = sorted(sim_scores, reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [36]:
1+1

2

In [37]:
recom = get_recommendations2('my love')
res = dict()
for i, v in recom.items():
    temp = ' '.join(v.split())
    if temp != '':
        res[i] = temp




In [38]:
#content based 没有考虑到用户的推荐
res

{1674: 'only girl in the world',
 1673: 'getting stronger',
 1672: 'weirdo you',
 1671: 'everglow',
 1670: 'everglow',
 1669: 'adventure of a lifetime',
 1668: 'up up',
 1667: 'hymn for the weekend',
 1666: 'the greatest',
 1665: 'april',
 1661: 'smells like teen spirit',
 1660: 'africa',
 1659: 'how deep is your love',
 1658: 'beat it body and soul',
 1653: 'i ll get used to it'}

In [39]:
def weighted_rating2(x, C, m):
    v = x['play_count']
    R = x['repeat_percentage']
    weight = (v/(v+m) * R) + (m/(m+v) * C)
    return weight


def improved_recommendations(title):
    idx = get_index_by_name(title)
    
    sim_scores = get_cosine_sim_scores(idx)
    sim_scores = sorted(sim_scores, reverse=True)
    sim_scores = sim_scores[1:26]
    music_indices = [i[0] for i in sim_scores]
    
    
    
    movies = train_features.iloc[music_indices][['name', 'play_count', 'repeat_percentage', 'song_year']]
    
    
    vote_counts = movies[movies['play_count'].notnull()]['play_count'].astype('int')
    vote_averages = movies[movies['repeat_percentage'].notnull()]['repeat_percentage'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['play_count'] >= m) & (movies['play_count'].notnull()) & (movies['repeat_percentage'].notnull())]
    qualified['play_count'] = qualified['play_count'].astype('int')
    qualified['repeat_percentage'] = qualified['repeat_percentage'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: weighted_rating2(x, C, m), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [40]:
im_recom = improved_recommendations('my love')

In [41]:
im_recom = pd.DataFrame(im_recom)
im_recom.dropna(axis=0, how='any', inplace=True)

In [42]:
im_recom[im_recom['name']!=' ']#·这里有bug，需要搞一搞”“这种情况

,name,play_count,repeat_percentage,song_year,wr
1672,weirdo you,1982,63,3.0,60.611253
1666,the greatest,1902,55,3.0,53.747790
1673,getting stronger,1851,55,3.0,53.718680
1654,,652,56,19.0,52.829755
1667,hymn for the weekend,753,52,3.0,50.373280
1674,only girl in the world,643,52,2.0,50.191763
1657,,1300,49,3.0,48.540930
1653,i ll get used to it,611,47,1.0,46.928119
1671,everglow,356,37,3.0,41.807441


In [43]:
def get_number(string):
    a = filter(lambda ch: ch in '0123456789', string)
    temp = ''.join(a)
    if len(temp) > 9:
        return temp[:7]
    else:
        return temp

train_features['msno_id'] = train_features.msno.apply(lambda x:get_number(x))
train_features['song_int_id'] = train_features.song_id.apply(lambda x:get_number(x))

In [44]:
train_features.head(5)

,index,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length_x,genre_ids_x,artist_name_x,composer_x,lyricist_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,description,msno_id,song_int_id
0,0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,local playlist more,local-library,224130.0,458,rachel liang,Qi Zheng Zhang,NaN,3.0,,TWUM71400047,1,0,NaN,7,2016-02-19,2017-09-18,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0,rachel liang,8773516,1725
1,1,1,1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,local playlist more,local-library,320470.0,465,jj lin,林俊傑,孫燕姿/易家揚,3.0,,TWB671005201,1,0,NaN,7,2016-02-19,2017-09-18,3804.0,6090.0,62.5,67621.0,115325.0,52.0,142.0,36.6,2.0,jj lin,8773516,9752253
2,2,2,2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,nan,song-based-playlist,315899.0,2022,yu takahashi,Yu Takahashi,Yu Takahashi,17.0,subarashiki nichijo,JPWP01070260,1,0,NaN,4,2016-11-17,2016-11-24,2.0,5.0,40.0,286.0,989.0,19.0,132.0,14.4,2.0,subarashiki nichijo yu takahashi,2270,8525304
3,3,3,3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,radio,radio,285210.0,465,u,The Edge| Adam Clayton| Larry Mullen| Jr.,NaN,52.0,hold me thrill me kiss me kill me,GBAAN0201228,3,30,male,9,2007-07-25,2017-04-30,7.0,31.0,22.6,228.0,698.0,115.0,388.0,29.6,0.0,hold me thrill me kiss me kill me u,164910,843549
4,4,4,4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,radio,radio,197590.0,873,yoga mr sound,Neuromancer,NaN,-1.0,om yoga,ITO101121898,3,30,male,9,2007-07-25,2017-04-30,2.0,5.0,40.0,59.0,180.0,115.0,388.0,29.6,2.0,om yoga yoga mr sound,164910,538


In [45]:
#train_features['repeat_percentage'] = train_features['repeat_percentage'].fillna(0)

In [46]:
#temp = train_features[['msno_id', 'id', 'repeat_percentage']]
#temp = temp.sort_values("msno_id",inplace=True)
#temp['repeat_percentage'] = temp['repeat_percentage'].fillna(0)

In [47]:
1+1

2

In [48]:
reader = Reader()
data = Dataset.load_from_df(train_features[['msno', 'song_id', 'repeat_percentage']], reader)

In [49]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25)

In [50]:
import gc
gc.collect()

22

In [51]:
svd = SVD()

In [52]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    45.5627 45.5641 45.5502 45.5934 45.5865 45.5714 0.0161  
MAE (testset)     42.2494 42.2659 42.2539 42.3227 42.2919 42.2768 0.0273  
Fit time          45.99   44.95   45.67   45.19   45.44   45.45   0.36    
Test time         2.00    2.49    2.01    2.00    2.51    2.20    0.24    


{'test_rmse': array([45.56274886, 45.56412308, 45.55020178, 45.59340832, 45.58653438]),
 'test_mae': array([42.24937344, 42.26588959, 42.25391275, 42.32269228, 42.29190033]),
 'fit_time': (45.98663878440857,
  44.954529762268066,
  45.6657829284668,
  45.186243295669556,
  45.435981035232544),
 'test_time': (2.004563570022583,
  2.489732027053833,
  2.006028890609741,
  1.9963715076446533,
  2.509911060333252)}

In [53]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [54]:
predictions = svd.test(testset)

In [55]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 45.5657


45.56567015886452

In [56]:
from surprise.model_selection import KFold

kf = KFold(n_splits=5)
algo = SVD()
for trainset, testset in kf.split(data):

    # 训练并测试算法
    algo.fit(trainset)
    predictions = algo.test(testset)

    # 计算并打印 RMSE（均方根误差，Root Mean Squared Error）
    accuracy.rmse(predictions, verbose=True)

RMSE: 45.5954
RMSE: 45.5674
RMSE: 45.5492
RMSE: 45.5940
RMSE: 45.5507


In [27]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(train_features, test_size=.25)

In [6]:
test_features['msno'].isna().sum()

0

In [49]:
train_features['source_system_tab'] = train_features['source_system_tab'].fillna('')
train_features['source_screen_name'] = train_features['source_screen_name'].fillna('')
train_features['source_type'] = train_features['source_type'].fillna('')
train_features['song_length_x'] = train_features['song_length_x'].fillna(0)
train_features['genre_ids_x'] = train_features['genre_ids_x'].fillna(0)
train_features['artist_name_x'] = train_features['artist_name_x'].fillna('')
train_features['lyricist_x'] = train_features['lyricist_x'].fillna('')
train_features['composer_x'] = train_features['composer_x'].fillna('')
train_features['language_x'] = train_features['language_x'].fillna(0)
train_features['name'] = train_features['name'].fillna('')
train_features['isrc'] = train_features['isrc'].fillna(0)
train_features['city'] = train_features['city'].fillna(0)
train_features['bd'] = train_features['bd'].fillna(0)
train_features['gender'] = train_features['gender'].fillna('')
train_features['registered_via'] = train_features['registered_via'].fillna(0)
train_features['registration_init_time'] = train_features['registration_init_time'].fillna('1901-01-01')
train_features['expiration_date'] = train_features['expiration_date'].fillna('1901-01-01')
train_features['repeat_count'] = train_features['repeat_count'].fillna(0)
train_features['play_count'] = train_features['play_count'].fillna(0)
train_features['repeat_percentage'] = train_features['repeat_percentage'].fillna(0)
train_features['repeat_count_artist'] = train_features['repeat_count_artist'].fillna(0)
train_features['play_count_artist'] = train_features['play_count_artist'].fillna(0)
train_features['repeat_count_msno'] = train_features['repeat_count_msno'].fillna(0)
train_features['play_count_msno'] = train_features['play_count_msno'].fillna(0)
train_features['repeat_percentage_msno'] = train_features['repeat_percentage_msno'].fillna(0)
train_features['song_year'] = train_features['song_year'].fillna(0)
#train_features['description'] = train_features['description'].fillna(0)

In [29]:
# test['play_count_msno'] = test['play_count_msno'].fillna(0)
# test['play_count_msno'] = test['play_count_msno'].astype('int')


# train['play_count_artist'] = train['play_count_artist'].fillna(0)
# test['play_count_artist'] = test['play_count_artist'].fillna(0)
# train['play_count_artist'] = train['play_count_artist'].astype('int')
# test['play_count_artist'] = test['play_count_artist'].astype('int')

In [20]:
drop_list = ['repeat_count','repeat_percentage','repeat_count_msno','repeat_percentage_msno']
drop_list2 = ['Unnamed: 0', 'Unnamed: 0.1']
train_features = train_features.drop(drop_list2,axis=1)
test_features = test_features.drop(drop_list2,axis=1)

In [21]:
for col in train_features.columns:
    if train_features[col].dtype == object:
        train_features[col] = train_features[col].astype('category')
        test_features[col] = test_features[col].astype('category')

In [16]:
train_features.shape

(7377418, 24)

In [19]:
# from lightfm import LightFM
# from lightfm.data import Dataset
# from lightfm.datasets import fetch_movielens
# from lightfm.evaluation import precision_at_k
# from lightfm import build_interactions

# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [22]:
test_features.head(1)

,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,song_length_x,genre_ids_x,artist_name_x,...,bd,gender,registered_via,registration_init_time,expiration_date,play_count,repeat_count_artist,play_count_artist,play_count_msno,song_year
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,local playlist more,224130.0,458,rachel liang,...,0,NaN,7,2016-02-19,2017-09-18,694.0,7102.0,13654.0,142.0,2.0


In [41]:
import gc
gc.collect()

111

In [11]:
# X = train_features.drop(['target'], axis=1)
# y = train_features['target'].values

In [12]:
# X_test = test_features.drop(['id'], axis=1)
# ids = test_features['id'].values

In [22]:
test_features.shape

(837941, 28)

In [23]:
train_features.shape

(7377418, 28)

In [24]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 28 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   msno                    category
 1   song_id                 category
 2   source_system_tab       category
 3   source_screen_name      category
 4   target                  int64   
 5   song_length_x           float64 
 6   genre_ids_x             category
 7   artist_name_x           category
 8   composer_x              category
 9   language_x              float64 
 10  name                    category
 11  isrc                    category
 12  city                    int64   
 13  bd                      int64   
 14  gender                  category
 15  registered_via          int64   
 16  registration_init_time  category
 17  expiration_date         category
 18  repeat_count            int64   
 19  play_count              int64   
 20  repeat_percentage       float64 
 21  repeat_c

In [25]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837941 entries, 0 to 837940
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   id                      837941 non-null  int64   
 1   msno                    837941 non-null  category
 2   song_id                 837941 non-null  category
 3   source_system_tab       835145 non-null  category
 4   source_screen_name      837941 non-null  category
 5   song_length_x           837934 non-null  float64 
 6   genre_ids_x             826839 non-null  category
 7   artist_name_x           837941 non-null  category
 8   composer_x              837941 non-null  category
 9   language_x              837925 non-null  float64 
 10  name                    837941 non-null  category
 11  isrc                    772937 non-null  category
 12  city                    837941 non-null  int64   
 13  bd                      837941 non-null  int64   
 14  gend

In [26]:
# drop_list3 = ['repeat_count', 'repeat_percentage_msno', 'repeat_count_msno','repeat_count','repeat_percentage']
# train_features = train_features.drop(drop_list3,axis=1)

In [27]:
train_features.to_csv('new_train_features.csv')
test_features.to_csv('new_test_features.csv')

In [40]:
train_features.drop(['description'], axis=1)
test_features.drop(['description'], axis=1)

KeyboardInterrupt: 

In [41]:
1

1

In [30]:
from sklearn.model_selection import KFold
# Create a Cross Validation with 3 splits
kf = KFold(n_splits=3)

predictions = np.zeros(shape=[len(test_features)])

# For each KFold
for train_indices ,validate_indices in kf.split(train_features) : 
    train_data = lgb.Dataset(train_features.drop(['target'],axis=1).loc[train_indices,:],label=train_features.loc[train_indices,'target'])
    val_data = lgb.Dataset(train_features.drop(['target'],axis=1).loc[validate_indices,:],label=train_features.loc[validate_indices,'target'])

    params = {
            'objective': 'binary',
            'boosting': 'gbdt',
            'learning_rate': 0.2 ,
            'verbose': 0,
            'num_leaves': 2**8,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 1,
            'feature_fraction': 0.9,
            'feature_fraction_seed': 1,
            'max_bin': 256,
            'num_rounds': 80,
            'metric' : 'auc'
        }
    # Train the model    
    lgbm_model = lgb.train(params, train_data, 100, valid_sets=[val_data])
    predictions += lgbm_model.predict(test_features.drop(['id'],axis=1))
    #del lgbm_model
    # We get the ammount of predictions from the prediction list, by dividing the predictions by the number of Kfolds.
predictions = predictions/3

[1]	valid_0's auc: 0.795423
[2]	valid_0's auc: 0.801305
[3]	valid_0's auc: 0.803781
[4]	valid_0's auc: 0.804801
[5]	valid_0's auc: 0.805937
[6]	valid_0's auc: 0.806517
[7]	valid_0's auc: 0.807163
[8]	valid_0's auc: 0.808295
[9]	valid_0's auc: 0.808176
[10]	valid_0's auc: 0.80833
[11]	valid_0's auc: 0.808354
[12]	valid_0's auc: 0.808465
[13]	valid_0's auc: 0.808013
[14]	valid_0's auc: 0.807852
[15]	valid_0's auc: 0.807576
[16]	valid_0's auc: 0.807391
[17]	valid_0's auc: 0.807248
[18]	valid_0's auc: 0.807174
[19]	valid_0's auc: 0.806776
[20]	valid_0's auc: 0.806601
[21]	valid_0's auc: 0.806255
[22]	valid_0's auc: 0.806195
[23]	valid_0's auc: 0.805969
[24]	valid_0's auc: 0.805574
[25]	valid_0's auc: 0.805252
[26]	valid_0's auc: 0.805025
[27]	valid_0's auc: 0.804842
[28]	valid_0's auc: 0.804571
[29]	valid_0's auc: 0.804509
[30]	valid_0's auc: 0.804369
[31]	valid_0's auc: 0.804064
[32]	valid_0's auc: 0.803772
[33]	valid_0's auc: 0.803747
[34]	valid_0's auc: 0.803532
[35]	valid_0's auc: 0.80

In [31]:
predictions[:10]

array([0.34299348, 0.34312911, 0.09279335, 0.05795618, 0.11693037,
       0.20485972, 0.19276682, 0.55738217, 0.10511296, 0.7035947 ])

In [46]:
sample = test_features.head(10)
sample

,id,msno,song_id,source_system_tab,source_screen_name,song_length_x,genre_ids_x,artist_name_x,composer_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,description
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,local playlist more,224130.0,458,rachel liang,qi zheng zhang,3.0,,TWUM71400047,1,0,NaN,7,2016-02-19,2017-09-18,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0,rachel liang
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,local playlist more,320470.0,465,jj lin,,3.0,,TWB671005201,1,0,NaN,7,2016-02-19,2017-09-18,3804.0,6090.0,62.5,67621.0,115325.0,52.0,142.0,36.6,2.0,jj lin
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,nan,315899.0,2022,yu takahashi,yu takahashi,17.0,subarashiki nichijo,JPWP01070260,1,0,NaN,4,2016-11-17,2016-11-24,2.0,5.0,40.0,286.0,989.0,19.0,132.0,14.4,2.0,subarashiki nichijo yu takahashi
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,radio,285210.0,465,u,the edge adam clayton larry mullen jr,52.0,hold me thrill me kiss me kill me,GBAAN0201228,3,30,male,9,2007-07-25,2017-04-30,7.0,31.0,22.6,228.0,698.0,115.0,388.0,29.6,0.0,hold me thrill me kiss me kill me u
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,radio,197590.0,873,yoga mr sound,neuromancer,-1.0,om yoga,ITO101121898,3,30,male,9,2007-07-25,2017-04-30,2.0,5.0,40.0,59.0,180.0,115.0,388.0,29.6,2.0,om yoga yoga mr sound
5,5,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,NV3nhEcMqsawwvSNTUAt9IVAexHLOm0lDfrHyEfN5B0=,radio,radio,216372.0,465,lee hi,nan,31.0,,TWA531300295,3,30,male,9,2007-07-25,2017-04-30,32.0,81.0,39.5,1895.0,3733.0,115.0,388.0,29.6,2.0,lee hi
6,6,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,DPM6G9RB5QO2dvAVPyc70gxHdvu872IjTUvV6LJT8ho=,radio,radio,200306.0,465,boa,joe belmaati,31.0,the shadow,KRA301200363,3,30,male,9,2007-07-25,2017-04-30,34.0,90.0,37.8,1804.0,4425.0,115.0,388.0,29.6,2.0,the shadow boa
7,7,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,JGXB3PHc0CX0JskwwjmYP8i318BLo7DhChgEj6Yqjt8=,my library,local playlist more,344305.0,458,steve chou,,3.0,,TWA470053008,3,30,male,9,2007-07-25,2017-04-30,628.0,1053.0,59.6,2235.0,4204.0,115.0,388.0,29.6,0.0,steve chou
8,8,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,507plIkmke1jh3wMrHqKore82pPFozADwydR8P0Gx2Q=,radio,radio,289332.0,465,ivy yan,,3.0,,CNR211200331,3,30,male,9,2007-07-25,2017-04-30,546.0,848.0,64.4,878.0,1485.0,115.0,388.0,29.6,2.0,ivy yan
9,9,3ZQ6oGfcSiUoCrtBPKGa8hHCiFh5jqtDqPVDUl/zrjU=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,local playlist more,232176.0,465,bigbang,nan,31.0,fantastic baby,SGA051200112,1,0,NaN,7,2015-12-19,2017-09-18,1424.0,2381.0,59.8,35317.0,61927.0,224.0,313.0,71.6,2.0,fantastic baby bigbang


In [47]:
sample = sample.drop(['id'],axis=1)

In [49]:
lgbm_model.predict(sample)

array([0.42033419, 0.45441297, 0.06985014, 0.04893676, 0.18739561,
       0.19036516, 0.16686494, 0.75222673, 0.09510204, 0.70536372])

In [36]:
# 模型保存
#lgbm_model.save_model('model.txt')

In [37]:
#gbm = lgb.Booster(model_file='model.txt')

In [38]:
gc.collect()

92

In [39]:
gbm.predict(sample)

array([0.42033419])